In [1]:
import numpy as np
sgm = lambda x: 1/(1+np.exp(-x))
nrand = np.random.random
zerov = np.zeros

In [42]:
class RBM:
    '''A restricted Boltzmann machine model.'''
    def __init__(self, n_variables, n_hidden, spin = False):
        self.a = zerov(n_variables)
        self.b = zerov(n_hidden)
        self.w = zerov((n_hidden, n_variables))
        self.min = -1*spin # Evaluates to zero if the model is not spin-based

    def activate(self, v, w, b, gap = 1):
        '''Turns the neurons of a forward layer on, according to a sigmoid probability function.'''
        vec = v@w + b # The neurons evaluation
        l = len(vec) # sample size
        prob = sgm(gap*vec)
        out = np.full(prob.shape,self.min)
        out[prob < nrand(prob.shape)] = 1
        return out
    
    def sample(self, v_data, steps = 1):
        h_data = self.activate(v_data, self.w.T, self.b)
        h_model = h_data
        for _ in range(steps):
            v_model = self.activate(h_model, self.w, self.a)
            h_model = self.activate(v_model, self.w.T, self.b)
        return h_data, v_model, h_model
    
    def grad(self, v_data, h_data, v_model, h_model):
        da = np.average(v_data - v_model, axis = 0)
        db = np.average(h_data - h_model, axis = 0)
        dw = sum(np.outer(v_data[i],h_data[i])-np.outer(v_model[i],h_model[i]) for i in range(len(v_data)))/len(v_data)
        #np.average(np.outer(v_data,h_data)-np.outer(v_model,h_model), axis = 1)
        return da, db, dw

    def train(self, data, n_epochs = 10, batch_size = 30, learning_rate = 1):
        batches = len(data)//batch_size
        for epoch in range(n_epochs):
            np.random.shuffle(data)
            for i in range(batches):
                v = data[i*batch_size:(i+1)*batch_size]
                h, v_m, h_m = self.sample(v)
                ca, cb, cw = self.grad(v,h,v_m,h_m)
                self.a += ca
                self.b += cb
                self.w += cw.T
            learning_rate = learning_rate/(0.05*learning_rate+1)



In [43]:
data = np.loadtxt('DATA/dataRBM_q0.1.csv', delimiter = ',')
rbm = RBM(len(data[0]),4)
rbm.train(data)

<ipython-input-1-1887cda36eef>:2: RuntimeWarning: overflow encountered in exp
  sgm = lambda x: 1/(1+np.exp(-x))
